# 0. Dependencies

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle
import pprint
from glob import glob
os.chdir('/home/users/alexno/sleep-staging')

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from h5py import File

# 1. Load data

In [3]:
df = pd.read_csv('overview_file_cohortsEM-ling1.csv')
# df['SubjectID'] = df['SubjectID'].apply(lambda row: row.split(' ')[0])
# df['ID'] = df['ID'].apply(lambda row: row.split(' ')[0])
# df['Diagnosis'] = df.apply(lambda x: x.Diagnosis.replace("'", ''), axis=1)
display(df.head())

,Index,ID,ID-ling,Cohort,Diagnosis,Label,Sleep scoring training data,Sleep scoring test data,Narcolepsy training data,Narcolepsy test data,Replication data,High pre-test,CSF hypocretin-1,DQ0602,Added by Alex
0,NaN,SSC_NARCO_1508_1,1508,SSC,'T1 NARCOLEPSY',1,0,1,1,0,0,1,na,1,1
1,NaN,SSC_NARCO_1534_1,1534,SSC,'T1 NARCOLEPSY',1,0,1,1,0,0,1,na,1,1
2,NaN,SSC_NARCO_1538_1,1538,SSC,'T1 NARCOLEPSY',1,0,1,0,1,0,1,na,1,1
3,NaN,SSC_NARCO_1647_1,1647,SSC,'T1 NARCOLEPSY',1,0,1,1,0,0,1,na,1,1
4,1267.0,SSC_1958_1,1958,SSC,'T1 NARCOLEPSY',1,0,1,1,0,0,1,0,0,0


In [4]:
selected_experiment = 'experiments/massc/avg_kw21/20201126_043826'
available_prediction_files = glob(os.path.join(selected_experiment, "*_results.csv"), recursive=True)
display(available_prediction_files)

['experiments/massc/avg_kw21/20201126_043826/SSC-WSC_more-spindles_results.csv',
 'experiments/massc/avg_kw21/20201126_043826/SSC-WSC_test_results.csv',
 'experiments/massc/avg_kw21/20201126_043826/KHC_results.csv',
 'experiments/massc/avg_kw21/20201126_043826/SSC-WSC_addspindles_results.csv',
 'experiments/massc/avg_kw21/20201126_043826/SSC-WSC_eval_results.csv']

In [5]:
df_preds = []
for result_file in [
#     'SSC-WSC_eval_results.csv', 
    'SSC-WSC_test_results.csv', 
    'KHC_results.csv'
]:
    df_preds.append(pd.read_csv(os.path.join(selected_experiment, result_file), index_col=0))
df_preds = pd.concat(df_preds).reset_index(drop=True)
df_preds = pd.merge(left=df_preds, right=df[['ID', 'Diagnosis', 'Label', 'Cohort']], how='inner', right_on='ID', left_on='SubjectID')
df_preds['Diagnosis'] = df_preds.apply(lambda x: x.Diagnosis.replace("'", ''), axis=1)
print(f"Unique classes: {df_preds['Diagnosis'].unique()}")
# print(f"Window unique classes: {df_preds['Window'].unique()}")
# print(f"Case unique classes: {df_preds['Case'].unique()}")

display(df_preds)

Unique classes: ['NON-NARCOLEPSY CONTROL' 'T1 NARCOLEPSY' 'OTHER HYPERSOMNIA']


,FileID,SubjectID,Window,Case,Accuracy,Balanced accuracy,Kappa,F1,Precision,Recall,...,Recall - N3,Support - N3,F1 - REM,Precision - REM,Recall - REM,Support - REM,ID,Diagnosis,Label,Cohort
0,A0014_7 031810.h5,A0014_7 031810,1 s,all,0.739486,0.795753,0.662931,0.706946,0.708011,0.795753,...,1.000000,78.0,0.895753,0.899225,0.892308,130.0,A0014_7 031810,NON-NARCOLEPSY CONTROL,0,WSC
1,A0019_4 171223.h5,A0019_4 171223,1 s,all,0.793617,0.747093,0.682385,0.601531,0.625021,0.597674,...,0.000000,0.0,0.930556,0.905405,0.957143,140.0,A0019_4 171223,NON-NARCOLEPSY CONTROL,0,WSC
2,A0032_7 172144.h5,A0032_7 172144,1 s,all,0.837780,0.836026,0.720180,0.756407,0.750209,0.836026,...,1.000000,34.0,0.931818,0.964706,0.901099,91.0,A0032_7 172144,NON-NARCOLEPSY CONTROL,0,WSC
3,A0036_7 090210.h5,A0036_7 090210,1 s,all,0.594866,0.667690,0.426980,0.453917,0.465531,0.667690,...,1.000000,8.0,0.558511,0.621302,0.507246,207.0,A0036_7 090210,NON-NARCOLEPSY CONTROL,0,WSC
4,A0046_3 170031.h5,A0046_3 170031,1 s,all,0.787657,0.848094,0.651418,0.644424,0.644282,0.848094,...,1.000000,4.0,0.942222,0.946429,0.938053,113.0,A0046_3 170031,NON-NARCOLEPSY CONTROL,0,WSC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,16584727.h5,16584727,1 s,all,0.579224,0.568724,0.461895,0.519832,0.522173,0.568724,...,0.880952,210.0,0.743772,0.657233,0.856557,244.0,16584727,OTHER HYPERSOMNIA,0,KHC
716,16683521.h5,16683521,1 s,all,0.610153,0.625956,0.511358,0.630171,0.658041,0.625956,...,0.795455,132.0,0.840376,0.917949,0.774892,231.0,16683521,OTHER HYPERSOMNIA,0,KHC
717,16748951.h5,16748951,1 s,all,0.575816,0.530146,0.377477,0.475312,0.485361,0.530146,...,0.333333,9.0,0.744186,0.793388,0.700730,137.0,16748951,OTHER HYPERSOMNIA,0,KHC
718,16788678.h5,16788678,1 s,all,0.569869,0.504916,0.409747,0.483004,0.480334,0.504916,...,0.826316,190.0,0.580645,0.592105,0.569620,158.0,16788678,OTHER HYPERSOMNIA,0,KHC


# 2. Analysis

In [6]:
display(df_preds['Cohort'].value_counts())
display(df_preds.groupby('Cohort')['Diagnosis'].value_counts(normalize=False))
display(df_preds.groupby('Cohort')['Diagnosis'].value_counts(normalize=True))
# display(df_preds.groupby('Cohort').agg({'Diagnosis': ['count']}))
display(df_preds.groupby('Cohort').agg({'Accuracy': ['mean', 'std']}))
display(df_preds.groupby('Diagnosis').agg({'Accuracy': ['mean', 'std']}))
display(df_preds.groupby(['Cohort', 'Diagnosis']).agg({'Accuracy': ['mean', 'std']}))

WSC    299
SSC    277
KHC    144
Name: Cohort, dtype: int64

Cohort  Diagnosis             
KHC     OTHER HYPERSOMNIA          69
        T1 NARCOLEPSY              63
        NON-NARCOLEPSY CONTROL     12
SSC     NON-NARCOLEPSY CONTROL    240
        T1 NARCOLEPSY              32
        OTHER HYPERSOMNIA           5
WSC     NON-NARCOLEPSY CONTROL    299
Name: Diagnosis, dtype: int64

Cohort  Diagnosis             
KHC     OTHER HYPERSOMNIA         0.479167
        T1 NARCOLEPSY             0.437500
        NON-NARCOLEPSY CONTROL    0.083333
SSC     NON-NARCOLEPSY CONTROL    0.866426
        T1 NARCOLEPSY             0.115523
        OTHER HYPERSOMNIA         0.018051
WSC     NON-NARCOLEPSY CONTROL    1.000000
Name: Diagnosis, dtype: float64

Accuracy          
            mean       std
Cohort                    
KHC     0.553565  0.104494
SSC     0.721244  0.096172
WSC     0.783001  0.067006

Accuracy          
                            mean       std
Diagnosis                                 
NON-NARCOLEPSY CONTROL  0.755939  0.088192
OTHER HYPERSOMNIA       0.584926  0.099655
T1 NARCOLEPSY           0.566400  0.118603

Accuracy          
                                   mean       std
Cohort Diagnosis                                 
KHC    NON-NARCOLEPSY CONTROL  0.557450  0.158955
       OTHER HYPERSOMNIA       0.573490  0.089233
       T1 NARCOLEPSY           0.531001  0.104910
SSC    NON-NARCOLEPSY CONTROL  0.732150  0.087608
       OTHER HYPERSOMNIA       0.742740  0.111768
       T1 NARCOLEPSY           0.636092  0.114373
WSC    NON-NARCOLEPSY CONTROL  0.783001  0.067006

In [126]:
print('Accuracy:')
print(f" Overall: \t{df_preds['Accuracy'].mean():.3f} + {df_preds['Accuracy'].std():.3f}")
print(f" WSC: \t\t{df_preds.loc[df_preds.Cohort == 'WSC']['Accuracy'].mean():.3f} + {df_preds.loc[df_preds.Cohort == 'WSC']['Accuracy'].std():.3f}")
print(f" SSC+KHC, no narcolepsy: \t\t{df_preds.loc[df_preds.Cohort == 'WSC']['Accuracy'].mean():.3f} + {df_preds.loc[df_preds.Cohort == 'WSC']['Accuracy'].std():.3f}")
display(df_preds.query('Cohort == "WSC"')[['Accuracy']].describe())
display(df_preds.query('(Cohort == "SSC" or Cohort == "KHC") and Label == 0')[['Accuracy']].describe())
display(df_preds.query('(Cohort == "SSC" or Cohort == "KHC") and Label == 1')[['Accuracy']].describe())

Accuracy:
 Overall: 	0.713 + 0.121
 WSC: 		0.783 + 0.067


,Accuracy
count,299.000000
mean,0.783001
std,0.067006
min,0.544211
25%,0.746325
50%,0.792654
75%,0.830759
max,0.951432


,Accuracy
count,326.000000
mean,0.692300
std,0.114888
min,0.137750
25%,0.618445
50%,0.703148
75%,0.782093
max,0.939673


,Accuracy
count,95.000000
mean,0.566400
std,0.118603
min,0.293173
25%,0.485214
50%,0.576136
75%,0.650544
max,0.826625


In [56]:
df.query('ID == "A0028_5 163131"')

,Index,ID,ID-ling,Cohort,Diagnosis,Label,Sleep scoring training data,Sleep scoring test data,Narcolepsy training data,Narcolepsy test data,Replication data,High pre-test,CSF hypocretin-1,DQ0602,Added by Alex


In [86]:
display(df_preds.groupby('Cohort')['Diagnosis'].value_counts(normalize=False))
display(df_preds.groupby('Cohort')['Diagnosis'].value_counts(normalize=True))

Cohort  Diagnosis               
KHC     'OTHER HYPERSOMNIA'          69
        'T1 NARCOLEPSY'              63
        'NON-NARCOLEPSY CONTROL'     12
SSC     'NON-NARCOLEPSY CONTROL'    240
        'T1 NARCOLEPSY'              32
        'OTHER HYPERSOMNIA'           5
WSC     'NON-NARCOLEPSY CONTROL'    299
Name: Diagnosis, dtype: int64

Cohort  Diagnosis               
KHC     'OTHER HYPERSOMNIA'         0.479167
        'T1 NARCOLEPSY'             0.437500
        'NON-NARCOLEPSY CONTROL'    0.083333
SSC     'NON-NARCOLEPSY CONTROL'    0.866426
        'T1 NARCOLEPSY'             0.115523
        'OTHER HYPERSOMNIA'         0.018051
WSC     'NON-NARCOLEPSY CONTROL'    1.000000
Name: Diagnosis, dtype: float64

In [39]:
df_preds.query('SubjectID == "10813475_p100"')

,FileID,SubjectID,Window,Case,Accuracy,Balanced accuracy,Kappa,F1,Precision,Recall,...,Recall - N2,Support - N2,F1 - N3,Precision - N3,Recall - N3,Support - N3,F1 - REM,Precision - REM,Recall - REM,Support - REM
56,10813475_p100.h5,10813475_p100,1 s,all,0.688967,0.50694,0.463583,0.390654,0.393907,0.405552,...,0.757417,573.0,0.0,0.0,0.0,0.0,0.686649,0.65625,0.72,175.0
